<a href="https://colab.research.google.com/github/iambusra/complingproject/blob/main/Preprocess_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install xlsxwriter

import pandas as pd
import xlsxwriter
from google.colab import files
import io
import numpy as np

In [40]:
# uploading the lexicon which we will use for sentiment analysis

data = files.upload()

In [88]:
lexicon = pd.read_csv("Turkish-tr-NRC-VAD-Lexicon.txt", sep="\t", error_bad_lines=False)

In [7]:
# uploading news data

files.upload()

Saving data_clean.xlsx to data_clean.xlsx


In [89]:
news = pd.read_excel("data_clean.xlsx")

# removing the first two columns, which do not contain any relevant information

news = news.drop(['Unnamed: 0', 'Haber Başlıkları'], axis = 1)
print(news)

                                               Linkler                                           Metinler
0                                    [Google Haberler]                                                NaN
1    http://www.haberinadresi.com/bursa-da-kadin-su...  [Bursa’da kimliği belirsiz bir kişi, gözüne ke...
2    http://www.pusulagazetesi.com.tr/kadin-surucu-...  Kadın sürücü kaza yaptı. Zonguldak’ta Kapuz Ma...
3    http://www.pusulagazetesi.com.tr/kadin-surucu-...  Kadın sürücü ve annesi yaralandı, kaza anı kam...
4    http://www.pusulagazetesi.com.tr/kaza-yapan-ka...  Zonguldak’ın Ereğli ilçesinde sürücünün direks...
..                                                 ...                                                ...
226  https://www.sozcu.com.tr/2021/gundem/alkollu-k...  İzmir'in Konak ilçesinde, otomobili uygulama n...
227  https://www.sozcu.com.tr/2021/gundem/seyahat-i...  Edinilen bilgilere göre, ilçe dışından gelen v...
228  https://www.sozcu.com.tr/2021/gunun-icind

In [ ]:
# preprocessing function that we have written in the class.
# We are using Turkish stop words this time since our data is in Turkish.

import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('turkish'))

def PreprocessNews(text):
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokenized_text = nltk.word_tokenize(text)
  clean_text = [word for word in tokenized_text if word not in stopwords ]
  return(clean_text)

In [91]:
# to be ensure that every row in Metinler is of type str for preprocessing
news.Metinler=news.Metinler.astype(str)

In [92]:
# preprocessing our news bodies
news['Metinler'] = news.apply(lambda x: PreprocessNews(x['Metinler']),axis=1)

In [ ]:
# turning the column to a list
metinler = news["Metinler"].tolist()
print(metinler)

Let's find the words that are present both in the Lexicon and the news bodies.

In [ ]:
matching_words = []
for sentence in metinler:
  for word in sentence:
    if word in lexicon["Turkish-tr"].tolist():
      matching_words.append(word)

print(matching_words)

How many words in our news bodies will end up having sentiment values by virtue of being listed in the lexicon?

In [95]:
len(matching_words)

11477

We have 11477 tokens in news bodies which are listed in the lexicon.

And how many types?

In [99]:
len(np.unique(matching_words))

1206

We have 1206 types in news bodies that are in the Sentiment lexicon and this is without any morphological analyzer. We will incorporate an analyzer (as Büşra Marşan's or a better one) to increase this number.

This notebook was sort of an exploratory analysis to see if our lexicon will be useful or not. Based on the number of matching lexical items, this lexicon is seems promising at least for this project. Now, what we further need to do is to find the matching lexical items in our lexicon for all words (including the morphologically complex ones) in our data.

Then, we will move on to randomly separate our data so that we have different data sets for both training and testing phases.

(**we are confused if we should train the classifier based on sentiment values, i.e. the relevant features, and the categories (biased vs. neutral) that we provide it as part of the training OR do a logistic regression or some sort of a linear model to understand the effect of Sentiment values on determining the category of the article.**)

We will find sentiment values for the tokens in the training set that have a match in the lexicon and save them in a data frame.

Later, we will analyze if sentiment values of gender-biased news articles differ in any dimension from the values of gender-neutral ones. Our prediction is that they will be different at least in one dimension. We will build a classifer to test this hypothesis and try to classify the articles in the test data based on sentiment values. Since we will have already categorized these two different news article types before the analysis, we will be able to measure the accuracy/precision of the classifier.